<a href="https://colab.research.google.com/github/krystal826/Natural-Language-Processing/blob/main/Lab_11_Task03_HobbAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This Python codeis about coreference resolution using Hobb algorithm. The code was developed by Chris Ward. Source: https://github.com/cmward/hobbs

In [29]:
"""
Implementation of Hobbs' algorithm for pronoun resolution.
Chris Ward, 2014
"""

import sys
import nltk
from nltk.corpus import names
from nltk import Tree
import queue

# Labels for nominal heads

In [30]:
nominal_labels = ["NN", "NNS", "NNP", "NNPS", "PRP"]

def get_pos(tree, node):
    """ Given a tree and a node, return the tree position
    of the node. 
    """ 
    for pos in tree.treepositions():
        if tree[pos] == node:
            return pos

In [31]:
def get_dom_np(sents, pos):
    """ Finds the position of the NP that immediately dominates 
    the pronoun.
    Args:
        sents: list of trees (or tree) to search
        pos: the tree position of the pronoun to be resolved
    Returns:
        tree: the tree containing the pronoun
        dom_pos: the position of the NP immediately dominating
            the pronoun
    """
    # start with the last tree in sents
    tree = sents[-1]
    # get the NP's position by removing the last element from 
    # the pronoun's
    dom_pos = pos[:-1]
    return tree, dom_pos

In [32]:
def walk_to_np_or_s(tree, pos):
    """ Takes the tree being searched and the position from which 
    the walk up is started. Returns the position of the first NP
    or S encountered and the path taken to get there from the 
    dominating NP. The path consists of a list of tree positions.
    Args:
        tree: the tree being searched
        pos: the position from which the walk is started
    Returns:
        path: the path taken to get the an NP or S node
        pos: the position of the first NP or S node encountered
    """
    path = [pos]
    still_looking = True
    while still_looking:
        # climb one level up the tree by removing the last element
        # from the current tree position
        pos = pos[:-1]
        path.append(pos)
        # if an NP or S node is encountered, return the path and pos
        if "NP" in tree[pos].label() or tree[pos].label() == "S":
            still_looking = False
    return path, pos

# Breadth-first search

In [33]:
def bft(tree):
    """ Perform a breadth-first traversal of a tree.
    Return the nodes in a list in level-order.
    Args:
        tree: a tree node
    Returns:
        lst: a list of tree nodes in left-to-right level-order
    """
    lst = []
    q = queue.Queue()
    q.put(tree)
    while not q.empty():
        node = q.get()
        lst.append(node)
        for child in node:
            if isinstance(child, nltk.Tree):
                q.put(child)
    return lst

In [34]:
def count_np_nodes(tree):
    """ Function from class to count NP nodes.
    """
    np_count = 0
    if not isinstance(tree, nltk.Tree):
        return 0
    elif "NP" in tree.label() and tree.label() not in nominal_labels:
        return 1 + sum(count_np_nodes(c) for c in tree)
    else:
        return sum(count_np_nodes(c) for c in tree)

In [35]:
def check_for_intervening_np(tree, pos, proposal, pro):
    """ Check if subtree rooted at pos contains at least 
    three NPs, one of which is: 
        (i)   not the proposal,
        (ii)  not the pronoun, and 
        (iii) greater than the proposal
    Args:
        tree: the tree being searched
        pos: the position of the root subtree being searched
        proposal: the position of the proposed NP antecedent
        pro: the pronoun being resolved (string)
    Returns:
        True if there is an NP between the proposal and the  pronoun
        False otherwise
    """
    bf = bft(tree[pos])
    bf_pos = [get_pos(tree, node) for node in bf]

    if count_np_nodes(tree[pos]) >= 3:
        for node_pos in bf_pos:
            if "NP" in tree[node_pos].label() \
            and tree[node_pos].label() not in nominal_labels: 
                if node_pos != proposal and node_pos != get_pos(tree, pro):
                    if node_pos < proposal:
                        return True
    return False

In [36]:
def traverse_left(tree, pos, path, pro, check=1):
    """ Traverse all branches below pos to the left of path in a
    left-to-right, breadth-first fashion. Returns the first potential
    antecedent found. 
    
    If check is set to 1, propose as an antecedent any NP node 
    that is encountered which has an NP or S node between it and pos. 
    If check is set to 0, propose any NP node encountered as the antecedent.
    Args:
        tree: the tree being searched
        pos: the position of the root of the subtree being searched
        path: the path taked to get to pos
        pro: the pronoun being resolved (string)
        check: whether or not there must be an intervening NP 
    Returns:
        tree: the tree containing the antecedent
        p: the position of the proposed antecedent
    """
    # get the results of breadth first search of the subtree
    # iterate over them
    breadth_first = bft(tree[pos])

    # convert the treepositions of the subtree rooted at pos
    # to their equivalents in the whole tree
    bf_pos = [get_pos(tree, node) for node in breadth_first]
    
    if check == 1:
        for p in bf_pos:
            if p<path[0] and p not in path:
                if "NP" in tree[p].label() and match(tree, p, pro):
                    if check_for_intervening_np(tree, pos, p, pro) == True:
                        return tree, p

    elif check == 0:
        for p in bf_pos:
            if p<path[0] and p not in path:
                if "NP" in tree[p].label() and match(tree, p, pro):
                    return tree, p

    return None, None

In [37]:
def traverse_right(tree, pos, path, pro):
    """ Traverse all the branches of pos to the right of path p in a 
    left-to-right, breadth-first manner, but do not go below any NP 
    or S node encountered. Propose any NP node encountered as the 
    antecedent. Returns the first potential antecedent.
    Args:
        tree: the tree being searched
        pos: the position of the root of the subtree being searched
        path: the path taken to get to pos
        pro: the pronoun being resolved (string)
    Returns:
        tree: the tree containing the antecedent
        p: the position of the antecedent
    """
    breadth_first = bft(tree[pos])
    bf_pos = [get_pos(tree, node) for node in breadth_first]

    for p in bf_pos:
        if p>path[0] and p not in path:
            if "NP" in tree[p].label() or tree[p].label() == "S":
                if "NP" in tree[p].label() and tree[p].label() not in nominal_labels:
                    if match(tree, p, pro):
                        return tree, p
                return None, None

In [38]:
def traverse_tree(tree, pro):
    """ Traverse a tree in a left-to-right, breadth-first manner,
    proposing any NP encountered as an antecedent. Returns the 
    tree and the position of the first possible antecedent.
    Args:
        tree: the tree being searched
        pro: the pronoun being resolved (string)
    """
    # Initialize a queue and enqueue the root of the tree
    q = queue.Queue()
    q.put(tree)
    while not q.empty():
        node = q.get()
        # if the node is an NP, return it as a potential antecedent
        if "NP" in node.label() and match(tree, get_pos(tree,node), pro):
            return tree, get_pos(tree, node)
        for child in node:
            if isinstance(child, nltk.Tree):
                q.put(child)
    # if no antecedent is found, return None
    return None, None

In [39]:
def match(tree, pos, pro):
    """ Takes a proposed antecedent and checks whether it matches
    the pronoun in number and gender
    
    Args:
        tree: the tree in which a potential antecedent has been found
        pos: the position of the potential antecedent
        pro: the pronoun being resolved (string)
    Returns:
        True if the antecedent and pronoun match
        False otherwise
    """
    if number_match(tree, pos, pro) and gender_match(tree, pos, pro):
        return True
    return False

# Number agreement

In [40]:
def number_match(tree, pos, pro):
    """ Takes a proposed antecedent and pronoun and checks whether 
    they match in number.
    """
    m = {"NN":          "singular",
         "NNP":         "singular",
         "he":          "singular",
         "she":         "singular",
         "him":         "singular",
         "her":         "singular",
         "his":         "singular",
         "it":          "singular",
         "himself":     "singular",
         "herself":     "singular",
         "itself":      "singular",
         "NNS":         "plural",
         "NNPS":        "plural",
         "they":        "plural",
         "them":        "plural",
         "themselves":  "plural",
         "PRP":         None}
    
    # if the label of the nominal dominated by the proposed NP and 
    # the pronoun both map to the same number feature, they match 
    for c in tree[pos]:
        if isinstance(c, nltk.Tree) and c.label() in nominal_labels:
            if m[c.label()] == m[pro]:
                return True
    return False

# Gender agreement

In [41]:
def gender_match(tree, pos, pro):
    """ Takes a proposed antecedent and pronoun and checks whether
    they match in gender. Only checks for mismatches between singular
    proper name antecedents and singular pronouns.
    """
    male_names = (name.lower() for name in names.words('male.txt'))
    female_names = (name.lower() for name in names.words('female.txt'))
    male_pronouns = ["he", "him", "himself"]
    female_pronouns = ["she", "her", "herself"]
    neuter_pronouns = ["it", "itself"]
    
    for c in tree[pos]:
        if isinstance(c, nltk.Tree) and c.label() in nominal_labels:
            # If the proposed antecedent is a recognized male name,
            # but the pronoun being resolved is either female or
            # neuter, they don't match
            if c.leaves()[0].lower() in male_names:
                if pro in female_pronouns:
                    return False
                elif pro in neuter_pronouns:
                    return False
            # If the proposed antecedent is a recognized female name,
            # but the pronoun being resolved is either male or 
            # neuter, they don't match
            elif c.leaves()[0].lower() in female_names:
                if pro in male_pronouns:
                    return False
                elif pro in neuter_pronouns:
                    return False
            # If the proposed antecedent is a numeral, but the 
            # pronoun being resolved is not neuter, they don't match
            elif c.leaves()[0].isdigit():
                if pro in male_pronouns:
                    return False
                elif pro in female_pronouns:
                    return False

    return True

# Hobb algorithm

In [42]:
def hobbs(sents, pos):
    """ The implementation of Hobbs' algorithm.
    Args:
        sents: list of sentences to be searched
        pos: the position of the pronoun to be resolved
    Returns:
        proposal: a tuple containing the tree and position of the
            proposed antecedent
    """
    # The index of the most recent sentence in sents
    sentence_id = len(sents)-1
    # The number of sentences to be searched
    num_sents = len(sents)

    # Step 1: begin at the NP node immediately dominating the pronoun
    tree, pos = get_dom_np(sents, pos)

    # String representation of the pronoun to be resolved
    pro = tree[pos].leaves()[0].lower()

    # Step 2: Go up the tree to the first NP or S node encountered
    path, pos = walk_to_np_or_s(tree, pos)
    
    # Step 3: Traverse all branches below pos to the left of path
    # left-to-right, breadth-first. Propose as an antecedent any NP
    # node that is encountered which has an NP or S node between it and pos
    proposal = traverse_left(tree, pos, path, pro)

    while proposal == (None, None):
        
        # Step 4: If pos is the highest S node in the sentence, 
        # traverse the surface parses of previous sentences in order
        # of recency, the most recent first; each tree is traversed in
        # a left-to-right, breadth-first manner, and when an NP node is
        # encountered, it is proposed as an antecedent
        if pos == ():
            # go to the previous sentence
            sentence_id -= 1
            # if there are no more sentences, no antecedent found
            if sentence_id < 0:
                return None
            # search new sentence
            proposal = traverse_tree(sents[sentence_id], pro)
            if proposal != (None, None):
                return proposal

        # Step 5: If pos is not the highest S in the sentence, from pos,
        # go up the tree to the first NP or S node encountered. 
        path, pos = walk_to_np_or_s(tree, pos)
        
        # Step 6: If pos is an NP node and if the path to pos did not pass 
        # through the nominal node that pos immediately dominates, propose pos 
        # as the antecedent.
        if "NP" in tree[pos].label() and tree[pos].label() not in nominal_labels:
            for c in tree[pos]:
                if isinstance(c, nltk.Tree) and c.label() in nominal_labels:
                    if get_pos(tree, c) not in path and match(tree, pos, pro):
                        proposal = (tree, pos)
                        if proposal != (None, None):
                            return proposal

        # Step 7: Traverse all branches below pos to the left of path, 
        # in a left-to-right, breadth-first manner. Propose any NP node
        # encountered as the antecedent.
        proposal = traverse_left(tree, pos, path, pro, check=0)
        if proposal != (None, None):
            return proposal

        # Step 8: If pos is an S node, traverse all the branches of pos
        # to the right of path in a left-to-right, breadth-forst manner, but
        # do not go below any NP or S node encountered. Propose any NP node
        # encountered as the antecedent.
        if tree[pos].label() == "S":
            proposal = traverse_right(tree, pos, path, pro)
            if proposal != (None, None):
                return proposal

    return proposal

In [43]:
def resolve_reflexive(sents, pos):
    """ Resolves reflexive pronouns by going to the first S
    node above the NP dominating the pronoun and searching for
    a matching antecedent. If none is found in the lowest S
    containing the anaphor, then the sentence probably isn't 
    grammatical or the reflexive is being used as an intensifier.
    """
    tree, pos = get_dom_np(sents, pos)

    pro = tree[pos].leaves()[0].lower()

    # local binding domain of a reflexive is the lowest clause 
    # containing the reflexive and a binding NP
    path, pos = walk_to_s(tree, pos)

    proposal = traverse_tree(tree, pro)

    return proposal

In [44]:
def walk_to_s(tree, pos):
    """ Takes the tree being searched and the position from which 
    the walk up is started. Returns the position of the first S 
    encountered and the path taken to get there from the 
    dominating NP. The path consists of a list of tree positions.
    Args:
        tree: the tree being searched
        pos: the position from which the walk is started
    Returns:
        path: the path taken to get the an S node
        pos: the position of the first S node encountered
    """
    path = [pos]
    still_looking = True
    while still_looking:
        # climb one level up the tree by removing the last element
        # from the current tree position
        pos = pos[:-1]
        path.append(pos)
        # if an S node is encountered, return the path and pos
        if tree[pos].label() == "S":
            still_looking = False
    return path, pos

# Demo - parse trees are provided here (tree1 - tree6)
## Sentence 1: John said he likes dogs
## Coreference resolution: he

In [45]:
nltk.download('names')
tree1 = Tree.fromstring('(S (NP (NNP John) ) (VP (VBD said) (SBAR (-NONE- 0) \
    (S (NP (PRP he) ) (VP (VBD likes) (NP (NNS dogs) ) ) ) ) ) )')

print("Sentence 1: John said he likes dogs.")
print(tree1)
tree, pos = hobbs([tree1], (1,1,1,0,0))
print("\nProposed antecedent for 'he':", tree[pos], '\n')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
Sentence 1: John said he likes dogs.
(S
  (NP (NNP John))
  (VP
    (VBD said)
    (SBAR
      (-NONE- 0)
      (S (NP (PRP he)) (VP (VBD likes) (NP (NNS dogs)))))))

Proposed antecedent for 'he': (NP (NNP John)) 



## Sentence 2: John said Mary likes him
## Coreference resolution: him

In [46]:
tree2 = Tree.fromstring('(S (NP (NNP John) ) (VP (VBD said) (SBAR (-NONE- 0) \
        (S (NP (NNP Mary) ) (VP (VBD likes) (NP (PRP him) ) ) ) ) ) )')

print("Sentence 2: John said Mary likes him.")
print(tree2)
tree, pos = hobbs([tree2], (1,1,1,1,1,0))
print("\nProposed antecedent for 'him':", tree[pos], '\n')

Sentence 2: John said Mary likes him.
(S
  (NP (NNP John))
  (VP
    (VBD said)
    (SBAR
      (-NONE- 0)
      (S (NP (NNP Mary)) (VP (VBD likes) (NP (PRP him)))))))

Proposed antecedent for 'him': (NP (NNP John)) 



## Sentence 3: John saw a flashy hat at the store
## Sentence 4: He showed it to Terence
## Corereference resolution: he and it

In [47]:
tree3 = Tree.fromstring('(S (NP (NNP John)) (VP (VBD saw) (NP (DT a) \
        (JJ flashy) (NN hat)) (PP (IN at) (NP (DT the) (NN store)))))')
tree4 = Tree.fromstring('(S (NP (PRP He)) (VP (VBD showed) (NP (PRP it)) \
        (PP (IN to) (NP (NNP Terrence)))))')

print("Sentence 3: John saw a flashy hat at the store.")
print("Sentence 4: He showed it to Terence.")
#print(tree3)
#print(tree4)
tree, pos = hobbs([tree3,tree4], (1,1,0))
print("\nProposed antecedent for 'it':", tree[pos])
tree, pos = hobbs([tree3,tree4], (0,0))
print("Proposed antecedent for 'he':", tree[pos], '\n')

Sentence 3: John saw a flashy hat at the store.
Sentence 4: He showed it to Terence.

Proposed antecedent for 'it': (NP (DT a) (JJ flashy) (NN hat))
Proposed antecedent for 'he': (NP (NNP John)) 



## Sentence 5: John said Mary likes herself
## Coreference resolution: herself

In [48]:
tree5 = Tree.fromstring('(S (NP (NNP John) ) (VP (VBD said) (SBAR (-NONE- 0) \
        (S (NP (NNP Mary) ) (VP (VBD likes) (NP (PRP herself) ) ) ) ) ) )')

print("Sentence 5: John said Mary likes herself.")
print(tree5)
tree, pos = resolve_reflexive([tree5], (1,1,1,1,1,0))
print("\nProposed antecedent for 'herself':", tree[pos], '\n')

Sentence 5: John said Mary likes herself.
(S
  (NP (NNP John))
  (VP
    (VBD said)
    (SBAR
      (-NONE- 0)
      (S (NP (NNP Mary)) (VP (VBD likes) (NP (PRP herself)))))))

Proposed antecedent for 'herself': (NP (NNP Mary)) 



## Sentence 6: Judge Curry ordered the refunds to begin Feb. and said that he wouldn't entertain any appeals or other attempts to block his order by Commonwealth Edison.
## Coreference resolution: he

In [49]:
tree6 = Tree.fromstring("(S(NP-SBJ (NNP Judge) (NNP Curry))\
        (VP(VP(VBD ordered)(NP-1 (DT the) (NNS refunds))\
        (S(NP-SBJ (-NONE- *-1))(VP (TO to) (VP (VB begin)\
        (NP-TMP (NNP Feb.) (CD 1))))))(CC and)\
        (VP(VBD said)(SBAR(IN that)(S(NP-SBJ (PRP he))(VP(MD would)\
        (RB n't)(VP(VB entertain)(NP(NP (DT any) (NNS appeals))(CC or)\
        (NP(NP(JJ other)(NNS attempts)(S(NP-SBJ (-NONE- *))(VP(TO to)\
        (VP (VB block) (NP (PRP$ his) (NN order))))))(PP (IN by)\
        (NP (NNP Commonwealth) (NNP Edison)))))))))))(. .))")

print("Sentence 6: Judge Curry ordered the refunds to begin Feb. and said that he wouldn't entertain any appeals or other attempts to block his order by Commonwealth Edison.")
#print(tree5)
tree, pos = hobbs([tree6], (1,2,1,1,0,0))
print("\nProposed antecedent for 'he':", tree[pos], '\n')

Sentence 6: Judge Curry ordered the refunds to begin Feb. and said that he wouldn't entertain any appeals or other attempts to block his order by Commonwealth Edison.

Proposed antecedent for 'he': (NP-SBJ (NNP Judge) (NNP Curry)) 



# Demo on Coreference resolution. Input from textfile (demosents.txt). Don't forget to download the text file.
## First example. The sample input:
## Enter the file: demosents.txt
## Enter the pronoun to resolve: He

In [51]:
p = ["He", "he", "Him", "him", "She", "she", "Her", "her", "It", "it", "They", "they"]
r = ["Himself", "himself", "Herself", "herself", "Itself", "itself", "Themselves", "themselves"]
fname = input("Enter the file: ")
pro = input("Enter the pronoun to resolve: ")
with open(fname) as f:
    sents = f.readlines()
print(sents)
trees = [Tree.fromstring(s) for s in sents]
pos = get_pos(trees[-1], pro)
pos = pos[:-1]
if pro in p:
    tree, pos = hobbs(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])
elif pro in r:
    tree, pos = resolve_reflexive(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])

Enter the file: demosents.txt
Enter the pronoun to resolve: He
['(S (NP (NNP John)) (VP (VBD saw) (NP (DT a)(JJ flashy) (NN hat)) (PP (IN at) (NP (DT the) (NN store)))))\n', '(S (NP (PRP He)) (VP (VBD showed) (NP (PRP it))(PP (IN to) (NP (NNP Terrence)))))\n']
(S
  (NP (NNP John))
  (VP
    (VBD saw)
    (NP (DT a) (JJ flashy) (NN hat))
    (PP (IN at) (NP (DT the) (NN store))))) 

(S
  (NP (PRP He))
  (VP (VBD showed) (NP (PRP it)) (PP (IN to) (NP (NNP Terrence))))) 

Proposed antecedent for 'He': (NP (NNP John))


# Demo on Coreference resolution. Input from textfile (demosents.txt)
## Second example. The sample input:
## Enter the file: demosents.txt
## Enter the pronoun to resolve: it

In [52]:
p = ["He", "he", "Him", "him", "She", "she", "Her", "her", "It", "it", "They", "they"]
r = ["Himself", "himself", "Herself", "herself", "Itself", "itself", "Themselves", "themselves"]
fname = input("Enter the file: ")
pro = input("Enter the pronoun to resolve: ")
with open(fname) as f:
    sents = f.readlines()
trees = [Tree.fromstring(s) for s in sents]
pos = get_pos(trees[-1], pro)
pos = pos[:-1]
if pro in p:
    tree, pos = hobbs(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])
elif pro in r:
    tree, pos = resolve_reflexive(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])

Enter the file: demosents.txt
Enter the pronoun to resolve: it
(S
  (NP (NNP John))
  (VP
    (VBD saw)
    (NP (DT a) (JJ flashy) (NN hat))
    (PP (IN at) (NP (DT the) (NN store))))) 

(S
  (NP (PRP He))
  (VP (VBD showed) (NP (PRP it)) (PP (IN to) (NP (NNP Terrence))))) 

Proposed antecedent for 'it': (NP (DT a) (JJ flashy) (NN hat))


# Demo on Coreference resolution. Input from textfile (demorefl.txt). Don't forget to download the text file.
## Third example. The sample input:
## Enter the file: demorefl.txt
## Enter the pronoun to resolve: herself

In [53]:
p = ["He", "he", "Him", "him", "She", "she", "Her", "her", "It", "it", "They", "they", "His", "his"]
r = ["Himself", "himself", "Herself", "herself", "Itself", "itself", "Themselves", "themselves"]
fname = input("Enter the file: ")
pro = input("Enter the pronoun to resolve: ")
with open(fname) as f:
    sents = f.readlines()
trees = [Tree.fromstring(s) for s in sents]
#print(trees)
pos = get_pos(trees[-1], pro)
print(pos)
pos = pos[:-1]
if pro in p:
    tree, pos = hobbs(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])
elif pro in r:
    tree, pos = resolve_reflexive(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])

Enter the file: demorefl.txt
Enter the pronoun to resolve: herself
(1, 1, 1, 1, 1, 0, 0)
(S
  (NP (NNP John))
  (VP
    (VBD said)
    (SBAR
      (-NONE- 0)
      (S (NP (NNP Mary)) (VP (VBD likes) (NP (PRP herself))))))) 

Proposed antecedent for 'herself': (NP (NNP Mary))


# Exercises:
## Based on the program segment provided to you below, suppose that you are implementing coreference resolution for the following sentences:
### Sentence 1: Alice saw a beautiful cupcake in the patisserie window. She showed it to Bob.
###             Coreference resolution: She and it.
### Sentence 2: Bob opened up a new dealership last week. John took a look at the Ford Falcon in his lot.
###            Coreference resolution: his

In [59]:
p = ["He", "he", "Him", "him", "She", "she", "Her", "her", "It", "it", "They", "they", "His", "his"]
r = ["Himself", "himself", "Herself", "herself", "Itself", "itself", "Themselves", "themselves"]
fname = input("Enter the file: ")
pro = input("Enter the pronoun to resolve: ")
with open(fname) as f:
    sents = f.readlines()
trees = [Tree.fromstring(s) for s in sents]
#print(trees)
pos = get_pos(trees[-1], pro)
print(pos)
if pro in p:
    tree, pos = hobbs(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])
elif pro in r:
    tree, pos = resolve_reflexive(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])

Enter the file: Alice.txt
Enter the pronoun to resolve: She
(0, 0, 0)
(S
  (NP (NNP Alice))
  (VP
    (VBD saw)
    (NP (DT a) (JJ beautiful) (NN cupcake))
    (PP (IN in) (NP (DT the) (NN patisserie) (NN window))))) 

(S
  (NP (PRP She))
  (VP (VBD showed) (NP (PRP it)) (PP (IN to) (NP (NNP Bob))))) 

Proposed antecedent for 'She': (NP (NNP Alice))


### How to get the parse tree representation? 
#### 1) Parse the sentence using Stanford CoreNLP tool (https://corenlp.run/). Choose Constituency parsing. 
#### 2) Convert the parse tree (output from the parsing process) into a list as shown in filename demosents.txt or demorefl.txt. 
#### 3) Save the file.
#### 4) Submit the new txt file you created and this Python program into ULearn.


In [60]:
p = ["He", "he", "Him", "him", "She", "she", "Her", "her", "It", "it", "They", "they", "His", "his"]
r = ["Himself", "himself", "Herself", "herself", "Itself", "itself", "Themselves", "themselves"]
fname = input("Enter the file: ")
pro = input("Enter the pronoun to resolve: ")
with open(fname) as f:
    sents = f.readlines()
trees = [Tree.fromstring(s) for s in sents]
#print(trees)
pos = get_pos(trees[-1], pro)
print(pos)
if pro in p:
    tree, pos = hobbs(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])
elif pro in r:
    tree, pos = resolve_reflexive(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])

Enter the file: Alice.txt
Enter the pronoun to resolve: it
(1, 1, 0, 0)
(S
  (NP (NNP Alice))
  (VP
    (VBD saw)
    (NP (DT a) (JJ beautiful) (NN cupcake))
    (PP (IN in) (NP (DT the) (NN patisserie) (NN window))))) 

(S
  (NP (PRP She))
  (VP (VBD showed) (NP (PRP it)) (PP (IN to) (NP (NNP Bob))))) 

Proposed antecedent for 'it': (NP (DT a) (JJ beautiful) (NN cupcake))


In [61]:
p = ["He", "he", "Him", "him", "She", "she", "Her", "her", "It", "it", "They", "they", "His", "his"]
r = ["Himself", "himself", "Herself", "herself", "Itself", "itself", "Themselves", "themselves"]
fname = input("Enter the file: ")
pro = input("Enter the pronoun to resolve: ")
with open(fname) as f:
    sents = f.readlines()
trees = [Tree.fromstring(s) for s in sents]
#print(trees)
pos = get_pos(trees[-1], pro)
print(pos)
if pro in p:
    tree, pos = hobbs(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])
elif pro in r:
    tree, pos = resolve_reflexive(trees, pos)
    for t in trees:
        print(t, '\n')
    print("Proposed antecedent for '"+pro+"':", tree[pos])

Enter the file: Bob.txt
Enter the pronoun to resolve: his
(1, 3, 1, 0, 0)
(S
  (NP (NNP Bob))
  (VP
    (VBD opened)
    (PRT (RP up))
    (NP (DT a) (JJ new) (NN dealership))
    (NP-TMP (JJ last) (NN week)))) 

(S
  (NP (NNP John))
  (VP
    (VBD took)
    (NP (DT a) (NN look))
    (PP (IN at) (NP (DT the) (NNP Ford) (NNP Falcon)))
    (PP (IN in) (NP (PRP$ his) (NN lot))))) 

Proposed antecedent for 'his': (NP (NNP John))
